<a href="https://colab.research.google.com/github/ishgirwan/omdena_hdi/blob/master/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

In [ ]:
!nvidia-smi 

Sun Oct 11 19:19:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install rasterio
#pip install pytorch-lightning
!pip install pytorch-lightning-bolts
#!pip install albumentations
!pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade

In [ ]:
import numpy as np
import pandas as pd
import rasterio

import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, sampler
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.metrics import MeanAbsoluteError
#import albumentations as A

#from sklearn.preprocessing import MinMaxScaler    
from sklearn.metrics import r2_score
import random
from PIL import Image

import glob
import os

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
csv_path = '/content/drive/My Drive/omdena_hdi/hdi_with_geometry.csv'
root_dir = '/content/drive/My Drive/Landsat/'

In [ ]:
model =  torchvision.models.resnet18(pretrained=False, progress=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
model.conv1 = nn.Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
model.fc = nn.Sequential(nn.Linear(in_features=512, out_features=1000, bias=True), nn.Sigmoid())

In [ ]:
class MyDataset(Dataset):
    """
    Generate normalized, rescaled and transformed datasets
    """

    def __init__(self, dataset_type, transform=None):
        
        super().__init__()
        self.df = dataset_type
        self.transform = transform

    def __len__(self):
        
        return len(self.df)
    
    def __getitem__(self, idx):
       
        if torch.is_tensor(idx):
              idx = idx.tolist()

        # generate image sample
        image_path = self.df['image_path'].iloc[idx] 
        image_sample = rasterio.open(str(image_path), "r")
        bands = [i for i in range(1, image_sample.count+1)]
        image_sample = image_sample.read(bands)
        image_sample = image_sample.astype('float32')

        # generate hdi sample

        hdi_sample = self.df['HDI'].iloc[idx]

        # Normalize the image sample and rescale it between 0 and 1
        channel_list=[]
        #set a seed so the same transforms are applied to each channel

        seed = np.random.randint(1)
        for ch in range(image_sample.shape[0]):
            channel_mean = np.nanmean(image_sample[ch])
            channel_stdev = np.nanstd(image_sample[ch])
            image_sample[ch] = (image_sample[ch] - channel_mean)

            if channel_stdev != 0:

                # standardize
                image_sample[ch] = image_sample[ch] / channel_stdev
                
                # normalize
                image_sample[ch] = (image_sample[ch] - np.nanmin(image_sample[ch])) / (np.nanmax(image_sample[ch]) - np.nanmin(image_sample[ch]))
            
            #random.seed(seed)
            #im_ch = Image.fromarray(image_sample[ch]).copy()
            #channel_tsfm = self.transform(im_ch)
            #channel_list.append(channel_tsfm)

        #image_sample = torch.cat(channel_list)

        # convet nan to 0
        image_sample[np.isnan(image_sample)] = 0
        image_sample = self.transform(image_sample)
        #print('hdi_sample', hdi_sample)

        return [image_sample.permute(1, 0, 2),  hdi_sample.astype('float32')]
    

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

# default used by the Trainer
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=4,
    strict=False,
    verbose=True,
    mode='min')


# DEFAULTS used by the Trainer
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/My Drive/omdena/ckpt/model.ckpt',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min',
    prefix=''
)

lr_monitor = LearningRateMonitor(logging_interval='step')


In [ ]:
def my_collate(batch):

    max_wh = 0

    for item in batch:
        image = item[0]
        w = image.shape[1]
        h = image.shape[2]
        max_i = np.max([w, h])
        if max_i > max_wh:
            max_wh = max_i
    
    #print(max_wh)

    data = []

    for item in batch:
        image = item[0]
        rows = image.shape[1]
        cols = image.shape[2]
        rows_diff = max_wh - rows
        cols_diff = max_wh - cols
        cols_half = int(cols_diff / 2)
        rows_half = int(rows_diff / 2)
        padding = (cols_half, cols_diff-cols_half, rows_half, rows_diff-rows_half)
        image_pad = F.pad(image, padding, 'constant', 0)
        #print(image.shape)
        #print(image_pad.shape)
        #print(type(image_pad))
        data.append(image_pad)

    target = [item[1] for item in batch]
    return [data, target]

In [ ]:
class Model(pl.LightningModule):

    def __init__(self, model, batch_size=1, learning_rate=.001):
        super().__init__()
        self.learning_rate = learning_rate
        self.save_hyperparameters()
        self.model = model
        self.batch_size = batch_size

    def forward(self, x):
        x = self.model(x)
        #print('shape', x.shape)
        x = torch.mean(x, 1)
        return x

    def prepare_data(self):

        df = pd.read_csv(csv_path)
        df['image_path'] = root_dir + df['unique code'].astype(str) + '.tif' 
        df = df.sample(frac=1, random_state=1).reset_index(drop=True)
        # split the dataset
        train, validate, test = np.split(df, [int(.9*len(df)), int(.95*len(df))]) 

        # transforms
        train_transform = transforms.Compose([
                                #transforms.RandomHorizontalFlip(),
                                #transforms.RandomVerticalFlip(),
                                transforms.ToTensor()
                                ])

        validate_transform = transforms.Compose([
                                transforms.ToTensor()
                                ])
        # create datasets for training, validation and test
        self.train_dataset = MyDataset(dataset_type=train, transform=train_transform)
        self.validate_dataset = MyDataset(dataset_type=validate, transform=validate_transform)
        self.test_dataset = MyDataset(dataset_type=test, transform=validate_transform) 
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, self.batch_size, shuffle=True, num_workers=4, collate_fn=my_collate, pin_memory=True, drop_last=True)

    def val_dataloader(self):
        return DataLoader(self.validate_dataset, self.batch_size, num_workers=4,collate_fn=my_collate, pin_memory=True, drop_last=True) 

    def test_dataloader(self):
        return DataLoader(self.test_dataset, self.batch_size, num_workers=4, collate_fn=my_collate, pin_memory=True, drop_last=True) 

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_loss'}

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        y = torch.cuda.FloatTensor(y)
        y_hat = self(x)
        #print(y, y_hat)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    #def training_step_end(self, training_step_outputs):
    #    return training_step_outputs

    #def training_epoch_end(self, outputs) -> None:
    #    torch.stack([x["loss"] for x in outputs]).mean()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        y = torch.cuda.FloatTensor(y)
        y_hat = self(x)
        #print(y, y_hat)
        loss = F.mse_loss(y_hat, y)
        r2 = r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
        self.log('val_loss', loss)
        self.log('val_R-square', r2)
        #return {"loss": loss, 'R-square': r2_score}

    #def validation_epoch_end(self, outputs) -> None:
    #    torch.stack([x['loss'] for x in outputs]).mean()
    #    torch.stack([x['R-square'] for x in outputs]).mean()

    #def validation_epoch_end(self, outputs):
    #   avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    #  return {'val_loss': avg_loss}

    def test_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        y = torch.cuda.FloatTensor(y)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        r2 = r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
        self.log('val_loss', loss)
        self.log('val_R-square', r2)
        #return {"loss": loss, 'R-square': r2_score}
    

    def backward(self, loss, optimizer, optimizer_idx):
        loss.backward()

    def optimizer_step(self, current_epoch, batch_idx, optimizer, 
      optimizer_idx, second_order_closure=None, 
       on_tpu=False, using_native_amp=False, using_lbfgs=False):
        optimizer.step()


In [ ]:
# init model
model_one = Model(model, batch_size=16)
#from pytorch_lightning.core.memory import ModelSummary
#ModelSummary(model_one, mode='full')
from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger('/content/drive/My Drive/omdena/tb_logs', name='my_model')

#train
root_path = '/content/drive/My Drive/omdena'

In [ ]:
›

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11 M  


Epoch 7: val_loss reached 0.01404 (best 0.01404), saving model to /content/drive/My Drive/omdena/ckpt/model.ckpt.ckpt as top 1
train_loss_step│train_loss│val_loss│val_R-square│train_loss_epoch
─────────────────────────────────────────────────────────────────
0.010572428815066814│0.010325191542506218│0.014036795124411583│-0.27880701422691345│0.010325191542506218


Epoch 8: val_loss reached 0.01279 (best 0.01279), saving model to /content/drive/My Drive/omdena/ckpt/model.ckpt-v0.ckpt as top 1
train_loss_step│train_loss│val_loss│val_R-square│train_loss_epoch
─────────────────────────────────────────────────────────────────
0.010572428815066814│0.010325191542506218│0.014036795124411583│-0.27880701422691345│0.010325191542506218
0.013365150429308414│0.010328253731131554│0.0127946175634861│-0.08919218927621841│0.010328253731131554


Epoch     8: reducing learning rate of group 0 to 1.0000e-04.


Epoch 9: val_loss reached 0.01183 (best 0.01183), saving model to /content/drive/My Drive/omdena/ckpt/model.ckpt.ckpt as top 1
train_loss_step│train_loss│val_loss│val_R-square│train_loss_epoch
─────────────────────────────────────────────────────────────────
0.010572428815066814│0.010325191542506218│0.014036795124411583│-0.27880701422691345│0.010325191542506218
0.013365150429308414│0.010328253731131554│0.0127946175634861│-0.08919218927621841│0.010328253731131554
0.008953281678259373│0.010243446566164494│0.011834335513412952│-0.009212560951709747│0.010243446566164494


Epoch 10: val_loss reached 0.01177 (best 0.01177), saving model to /content/drive/My Drive/omdena/ckpt/model.ckpt-v0.ckpt as top 1
train_loss_step│train_loss│val_loss│val_R-square│train_loss_epoch
─────────────────────────────────────────────────────────────────
0.010572428815066814│0.010325191542506218│0.014036795124411583│-0.27880701422691345│0.010325191542506218
0.013365150429308414│0.010328253731131554│0.0127946175634861│-0.08919218927621841│0.010328253731131554
0.008953281678259373│0.010243446566164494│0.011834335513412952│-0.009212560951709747│0.010243446566164494
0.0027869234327226877│0.010170964524149895│0.011767010204494│-0.007069265004247427│0.010170964524149895


Epoch 11: val_loss reached 0.01175 (best 0.01175), saving model to /content/drive/My Drive/omdena/ckpt/model.ckpt.ckpt as top 1
train_loss_step│train_loss│val_loss│val_R-square│train_loss_epoch
─────────────────────────────────────────────────────────────────
0.010572428815066814│0.010325191542506218│0.014036795124411583│-0.27880701422691345│0.010325191542506218
0.013365150429308414│0.010328253731131554│0.0127946175634861│-0.08919218927621841│0.010328253731131554
0.008953281678259373│0.010243446566164494│0.011834335513412952│-0.009212560951709747│0.010243446566164494
0.0027869234327226877│0.010170964524149895│0.011767010204494│-0.007069265004247427│0.010170964524149895
0.011240934953093529│0.010136882774531841│0.011748147197067738│-0.0029800510965287685│0.010136882774531841


In [ ]:
!nvidia-smi 

In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir='/content/drive/My Drive/omdena/tb_logs'

In [ ]:
# test
trainer.test()